## Analysis3: Identify key conversations related to Fraud and Corporate Crime by looking for specific keywords

#### Step 0: Identify risky words

In [1]:
# A pool of words that are considered risky
# because they were either associated with misleading financial statements
# or they were associated with insider trading
risky_words = ['bankruptcy', 'bankrupt', 'stocks', 'shares', 'audit', 'enrn', 'ene', 'nyse', 'nasdaq'
               '401k', 'offshore', 'early', 'notice', 'billion', 'million', 'price', 'warning', 
              'overseas', 'international', 'mark-to-market', 'MTM']

#### Step 1: Import modules

In [2]:
from email.parser import Parser
import glob
import os
import datetime

#### Step 2: Identify directories

In [3]:
# Path to the Enron dataset
path_to_maildir = 'C:/Users/parit/midterm/data/enron/maildir'

In [4]:
# Path to primary convicts mailboxes (Andrew Fastow's mailbox is not present in the dataset)
path_to_kenneth_lay = path_to_maildir + '/lay-k'
path_to_jeff_skilling = path_to_maildir + '/skilling-j'


#### Step 3: Define processing logic

In [5]:
# Function to recursively read mails from folders and sub-folders
def processEmails(path):
    peak_mail_words = []
    dwnfall_mail_words = []
    inner_peak = []
    inner_dwnfall = []
    pathContent = glob.glob(path + '/*')    
    for pathName in pathContent:
        if os.path.isdir(pathName):    
            (inner_peak, inner_dwnfall) = processEmails(pathName)
            peak_mail_words = peak_mail_words + inner_peak
            dwnfall_mail_words = dwnfall_mail_words + inner_dwnfall
        else:
            try:
                with open(pathName, 'r') as filename:
                    email = Parser().parsestr(filename.read())                    
                    mail_date = email['date']
                    mail_subject = email['subject'].split()
                    mail_content = email.get_payload().split()                    
                    mail_words = mail_content + mail_subject
                    mail_date = datetime.datetime.strptime(mail_date[:-6],"%a, %d %b %Y %H:%M:%S %z")
                    if mail_date.year in range(1997,2001):                        
                        peak_mail_words = [r for r in mail_words if lower(r) in risky_words]
                    elif mail_date.year == 2001:
                        dwnfall_mail_words = [r for r in mail_words if lower(r) in risky_words]
            except:
                print('File open error due to foreign characters:', pathName)
    return (peak_mail_words, dwnfall_mail_words)

#### Step 4: Invoke email processing for Kenneth Lay

In [6]:
# Invoke processEmails() for Kenneth Lay
(before, after) = processEmails(path_to_kenneth_lay)
print('Risky words for Kenneth Lay in first phase:', before)
print('Risky words for Kenneth Lay in second phase:', after)

File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1000_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1001_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1002_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1003_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1004_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1005_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1006_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1007_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/lay-k\all_documents\1008_
F

#### Step 5: Invoke email processing for Jeff Skilling


In [7]:
# Invoke processEmails() for Jeff Skilling
(before, after) = processEmails(path_to_jeff_skilling)
print('Risky words for Jeff Skilling in first phase:', before)
print('Risky words for Jeff Skilling in second phase:', after)

File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\100_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\101_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\102_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\103_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\104_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\105_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\106_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/maildir/skilling-j\all_documents\107_
File open error due to foreign characters: C:/Users/parit/midterm/data/enron/mai

### Conclusion for Analysis 3: 

> ...